In [1]:
import pandas as pd
df = pd.read_csv('/content/sample_data/03_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [2]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.7 MB/s eta 0:00:00


In [3]:
%pip install lightbgm

ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


In [30]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMRegressor


stacking_reg = StackingRegressor(
    estimators=[
        ('lin_reg', LinearRegression()),
        ('LGBM_01', LGBMRegressor(n_estimators=100, max_depth=6, random_state=42)),
        ('LGBM_02', LGBMRegressor(n_estimators=60, max_depth=8, random_state=42)),
        ('xgboost_01', xgb.XGBRegressor(max_depth=6, n_estimators=60, random_state=42)),
        ('xgboost_02', xgb.XGBRegressor(max_depth=8, n_estimators=40, random_state=42)),
        ('catboost_01',Catboost(iterations=10, depth=10, learning_rate=1, random_state=42)),
        ('catboost_02',Catboost(iterations=20, depth=8, learning_rate=1, random_state=42)),
    ],
    final_estimator=Catboost(iterations=20, depth=8, learning_rate=1, random_state=42),
    n_jobs=-1,
    cv=3
)
stacking_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


0:	learn: 1.0584216	total: 173ms	remaining: 3.28s
1:	learn: 1.0580205	total: 339ms	remaining: 3.05s
2:	learn: 1.0577229	total: 492ms	remaining: 2.79s
3:	learn: 1.0572180	total: 705ms	remaining: 2.82s
4:	learn: 1.0570808	total: 994ms	remaining: 2.98s
5:	learn: 1.0568907	total: 1.26s	remaining: 2.94s
6:	learn: 1.0567510	total: 1.53s	remaining: 2.85s
7:	learn: 1.0565439	total: 1.8s	remaining: 2.7s
8:	learn: 1.0564138	total: 2.09s	remaining: 2.55s
9:	learn: 1.0563525	total: 2.35s	remaining: 2.35s
10:	learn: 1.0561825	total: 2.61s	remaining: 2.13s
11:	learn: 1.0560352	total: 2.88s	remaining: 1.92s
12:	learn: 1.0559572	total: 3.12s	remaining: 1.68s
13:	learn: 1.0557982	total: 3.4s	remaining: 1.46s
14:	learn: 1.0556934	total: 3.67s	remaining: 1.22s
15:	learn: 1.0554113	total: 3.95s	remaining: 987ms
16:	learn: 1.0552954	total: 4.21s	remaining: 743ms
17:	learn: 1.0551513	total: 4.5s	remaining: 500ms
18:	learn: 1.0549931	total: 4.79s	remaining: 252ms
19:	learn: 1.0548388	total: 5.06s	remaining: 

StackingRegressor(cv=3,
                  estimators=[('lin_reg', LinearRegression()),
                              ('LGBM_01',
                               LGBMRegressor(max_depth=6, random_state=42)),
                              ('LGBM_02',
                               LGBMRegressor(max_depth=8, n_estimators=60,
                                             random_state=42)),
                              ('xgboost_01',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_st...
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=40, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                              ('catboost_01',
                               <catboost.core.CatBoostRegressor object at 0x7c299a895a80>),
                              ('catboost_02',
                               <catboost.core.CatBoostRegressor object at 0x7c299a895120>)],
                  final_estimator=<catboost.core.CatBoostRegressor object at 0x7c299a897ee0>,
                  n_jobs=-1)

In [31]:
from sklearn.metrics import root_mean_squared_error
y_pred = stacking_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0541576799646193

In [32]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [33]:
y_pred = stacking_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0593905520544489

In [34]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/03_test.csv')
del test_set['Premium Amount']
y_pred = stacking_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('stacking_reg_03.csv', index=False)